In [ ]:
import pandas as pd
from biopandas.pdb import PandasPdb
import plotly.graph_objects as go
import numpy as np
import webbrowser

sasa_df = pd.read_excel(r"sasa_results.xlsx")
non_zero_sasa = sasa_df[sasa_df["SASA (Å²)"] > 0]["SASA (Å²)"]
threshold = non_zero_sasa.median() if len(non_zero_sasa) > 0 else 0.0
threshold = 35.596                  #阈值替换
print(f"Threshould: {threshold:.3f} Å²")

ppdb = PandasPdb().read_pdb(r"1ymg_chainA.pdb")
atom_df = ppdb.df['ATOM']


ca_df = atom_df[atom_df['atom_name'] == 'CA'].copy()
ca_df['residue_number'] = ca_df['residue_number'].astype(int)


sasa_dict = {(row['Chain'], row['R.No.']): row['SASA (Å²)'] 
             for _, row in sasa_df.iterrows()}


colors = []
surface_residues = []

for _, row in ca_df.iterrows():
    chain = row['chain_id']
    res_no = row['residue_number']
    res_name = row['residue_name']
    
    sasa = sasa_dict.get((chain, res_no), 0.0)
    
    if sasa > threshold:
        colors.append("red")
        surface_residues.append({
            "Chain": chain,
            "Residue Number": res_no,
            "Amino Acid": res_name,
            "SASA (Å²)": sasa
        })
    else:
        colors.append("black")


fig = go.Figure(data=[
    go.Scatter3d(
        x=ca_df['x_coord'],
        y=ca_df['y_coord'],
        z=ca_df['z_coord'],
        mode='markers',
        marker=dict(
            size=7.5,
            color=colors,
            opacity=0.8
        ),
        hovertext=[
            f"{row['residue_name']}{row['residue_number']} ({row['chain_id']})" 
            for _, row in ca_df.iterrows()
        ]
    )
])

fig.update_layout(
    title=f"AQP0 Surface AAs Visualization ( with Est.Threshold: {threshold:.3f}Å² )",
    scene=dict(
        xaxis_title='X (Å)',
        yaxis_title='Y (Å)',
        zaxis_title='Z (Å)',
        aspectmode='data'
    ),
    width=1200,
    height=1000
)

fig.show()
fig.write_html(r"AQP0Surface_AAs_3D_View.html")
webbrowser.open(r"AQP0Surface_AAs_3D_View.html")

surface_df = pd.DataFrame(surface_residues)
surface_df = surface_df.sort_values(by=["Chain", "Residue Number"])

output_path = r"Surface_AAs.xlsx"
surface_df.to_excel(output_path, index=False)
print(f"Results already saved to: {output_path}")